In [28]:
import pandas as pd
import json

In [37]:
def data_extraction(file_path):
    data = json.load(open(file_path))
    data = data['data']
    new_data = []
    for example in data:
        for paragraph in example['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                answers = qa['answers']
                for answer in answers:
                    new_data.append({'context': context, 'question': question, 'answer': answer['text'], 'answer_start': answer['answer_start'],'title': example['title']})
    return new_data

In [38]:
data = data_extraction('../MLQA_V1/dev/dev-context-vi-question-vi.json')

In [39]:
data.extend(data_extraction('../MLQA_V1/test/test-context-vi-question-vi.json'))

In [40]:
df = pd.DataFrame(data)

In [80]:
df.to_csv('data_gen/MLQA_V1_vi.csv', index=False)